# KNN Implementation From Scratch
    Implement the K-Nearest neighbors algorithm from scratch 

In [1]:
# requierments
import pandas as pd
import numpy as np
import operator
import random
import matplotlib.pyplot as plt

# Implementations
* calculate distance between data points by method euclidean, hamming, max_distance
* Find nearest neighbors
* Find K-nearest neighbors
* Calculate majority votes by method random, weighted
* KNN & Weighted KNN

In [ ]:
path_datests = './datasets'
path_Q4_data = '{data}/Question 4/'.format(data = path_datests)

def read_csv_datasets(train_data_name, test_data_name):
    train_data = pd.read_csv(path_Q4_data + train_data_name, header=None)
    test_data = pd.read_csv(path_Q4_data + test_data_name, header=None)
    return [train_data, test_data]

def distance_calculator(vec1, vec2, how = 'euclidean'):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)

    if (how == 'euclidean'): 
        return np.linalg.norm(vec1 - vec2)
    elif(how == 'd1'): 
        return np.max(vec1 - vec2)
    elif(how == 'hamming'):
        return np.sum(np.abs(vec1 - vec2))
    else:
        return
    
def KNN(training_set, training_labels, test_instance, k, how_mesure_distance = 'euclidean', optimized = True):

    distances = {}
    if(optimized):
        if(how_mesure_distance == 'euclidean'):
            distances = (training_set -  test_instance).T.apply(np.linalg.norm)
        elif(how_mesure_distance == 'd1'):
            distances = (training_set -  test_instance).T.apply(np.max)
        elif(how_mesure_distance == 'hamming'):
            distances = (training_set -  test_instance).T.apply(np.abs).apply(np.sum)
        
        distances = dict(distances)
        
    else:
        for row in training_set.itertuples():
            distances[row[0]] = distance_calculator(np.array(row[1:]), np.array(test_instance), how = 'euclidean')

    
    sorted_distances = {j: v for j, v in sorted(distances.items(), key=lambda item: item[1])}

    
    dist_of_NNs = list(sorted_distances.values())[:k]
    ids_of_NNs = list(sorted_distances.keys())[:k]
    classes_of_NNs = list(training_labels.iloc[ids_of_NNs].iloc[:,0])
    
    
    K_nearest_neighbours = {}
    K_nearest_neighbours['class'] = classes_of_NNs
    K_nearest_neighbours['distance'] = dist_of_NNs
    
    
    return pd.DataFrame(K_nearest_neighbours)

def random_majority_vote(nearest_neighbours):
    class_votes = dict(nearest_neighbours['class'].value_counts())
    max_vote = max(class_votes.values())
    top_votes = [k for k,v in class_votes.items() if v == max_vote]
    if(len(top_votes) < 2):
        return top_votes[0]
    else:
        return random.choice(top_votes)
    
def weighted_majority_vote(nearest_neighbours):
    weighted_class_votes = {}
    for class_name,group in nearest_neighbours.groupby('class'):
        weighted_class_votes[class_name] = 0
        for i, row in group.iterrows():
            weighted_class_votes[class_name] += (1/row['distance'])
        
    max_vote_class = (max(weighted_class_votes.keys(), key=(lambda key: weighted_class_votes[key])))
    
    return max_vote_class
    
def majority_vote(nearest_neighbours, break_ties_method = 'weighted'):

    if(break_ties_method == 'random'):
        return random_majority_vote(nearest_neighbours)
        
    elif(break_ties_method == 'weighted'):
        return weighted_majority_vote(nearest_neighbours)
    
def predict_test_data_KNN(training_set, training_labels, test_set, k = 10, how_mesure_distance = 'euclidean', break_ties_method = 'random'):
    predictions = []
    
    for test_instance in test_set.itertuples():
        
        nearest_neighbors = KNN(training_set, training_labels, np.array(test_instance[1:]),
                                k = k,
                                how_mesure_distance = how_mesure_distance)
        predicted_class = majority_vote(nearest_neighbors,
                                        break_ties_method = break_ties_method)
        predictions.append(predicted_class)
    return(pd.Series(predictions))

def calculate_accuracy(test, prediction):
    try:
        return (sum(test == prediction)/len(prediction))
    except:
        return pd.NA
    
def normalize_dataset(data):
    try:
        return (data - data.min()) / (data.max() - data.min())
    except:
        return pd.NA
    

In [ ]:
train_data, test_data = read_csv_datasets('train_data.csv', 'test_data.csv')
train_label, test_label = read_csv_datasets('train_labels.csv', 'test_labels.csv')
train_data_normalized = normalize_dataset(train_data)
test_data_normalized = normalize_dataset(test_data)

In [ ]:
Ks = [2,5,10,50]
accuracies = []
for k in Ks:
    predictions = predict_test_data_KNN(train_data, train_label, test_data, k = k, how_mesure_distance = 'euclidean', break_ties_method = 'random')
    accuracies.append(calculate_accuracy(test_label.iloc[:,0], predictions))

In [ ]:
ax = pd.Series(data=accuracies[4:], index=Ks).plot(kind = 'line', linestyle='-',
                                          marker='o',
                                          title = 'Accuracy ~ Neighbor Numbers on normalized dataset test set')
ax.set_xlabel("Neighbor Numbers")
ax.set_ylabel("Accuracy")

In [ ]:
best_k = 5
predictions = predict_test_data_KNN(train_data_normalized, train_label, test_data_normalized, k = best_k, how_mesure_distance = 'hamming', break_ties_method = 'random')
accuracy_hamming = (calculate_accuracy(test_label.iloc[:,0], predictions))
accuracy_hamming

In [ ]:
best_k = 5
predictions = predict_test_data_KNN(train_data_normalized, train_label, test_data_normalized, k = best_k, how_mesure_distance = 'd1', break_ties_method = 'random')
accuracy_d1 = (calculate_accuracy(test_label.iloc[:,0], predictions))
accuracy_d1